# Piper TTS

In [ ]:
!pip install -q --upgrade gradio piper-tts

In [ ]:
import gradio as gr
import tempfile
import numpy as np
import soundfile as sf
from pathlib import Path

from piper.voice import PiperVoice
from piper.download import get_voices, ensure_voice_exists, find_voice

# 1. Directorio donde Piper guarda las voces
download_dir = Path.home() / ".local/share/piper-tts/piper-voices"
voices_info = get_voices(download_dir)

# 2. Mapeo de idiomas → nombre de voz small, masculina
voice_names = {
    "Español": "es_ES-carlfm-x_low",
    "English": "en_US-ryan-low"
}

# 3. Elimina configs corruptas para forzar redescarga
for name in voice_names.values():
    cfg = download_dir / f"{name}.onnx.json"
    if cfg.exists():
        cfg.unlink()

# 4. Descarga (si falta) y carga cada modelo en un dict
voices = {}
for lang, name in voice_names.items():
    ensure_voice_exists(name, [download_dir], download_dir, voices_info)
    model_path, config_path = find_voice(name, [download_dir])
    voices[lang] = PiperVoice.load(model_path, config_path=config_path)

# 5. Función TTS con firma corregida: (idioma, texto)
def tts(idioma: str, texto: str) -> str:
    voice = voices[idioma]
    raw_chunks = voice.synthesize_stream_raw(texto)
    audio_bytes = b"".join(raw_chunks)
    audio_array = np.frombuffer(audio_bytes, dtype=np.int16)

    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp.name, audio_array, voice.config.sample_rate, subtype="PCM_16")
    return tmp.name

# 6. Interfaz Gradio: primero dropdown → segundo textbox
iface = gr.Interface(
    fn=tts,
    inputs=[
        gr.Dropdown(choices=list(voice_names.keys()),
                    value="Español",
                    label="Idioma"),
        gr.Textbox(lines=3,
                   placeholder="Escribe tu texto aquí...",
                   label="Texto")
    ],
    outputs=gr.Audio(type="filepath"),
    title="TTS multilenguaje con Piper y Gradio",
    description="Selecciona idioma, escribe un texto y escucha la síntesis de voz (modelos small, voces masculinas)."
)

if __name__ == "__main__":
    # En Colab se auto-activa share=True, si no lo quieres pon share=False
    iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d79a8d7c0911d4bdd7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d79a8d7c0911d4bdd7.gradio.live
